[download this notebook here](https://github.com/HumanCompatibleAI/imitation/blob/master/docs/tutorials/4_train_airl.ipynb)
# Train an Agent using Adversarial Inverse Reinforcement Learning

As usual, we first need an expert. Again, we download one from the HuggingFace model hub for convenience.

Note that we now use a variant of the CartPole environment from the seals package, which has fixed episode durations. Read more about why we do this [here](https://imitation.readthedocs.io/en/latest/getting-started/variable-horizon.html).

In [1]:
import numpy as np
from imitation.policies.serialize import load_policy
from imitation.util.util import make_vec_env
from imitation.data.wrappers import RolloutInfoWrapper

SEED = 42

FAST = False

if FAST:
    N_RL_TRAIN_STEPS = 100_000
else:
    N_RL_TRAIN_STEPS = 2_000_000

venv = make_vec_env(
    "seals:seals/CartPole-v0",
    rng=np.random.default_rng(SEED),
    n_envs=8,
    post_wrappers=[
        lambda env, _: RolloutInfoWrapper(env)
    ],  # needed for computing rollouts later
)
expert = load_policy(
    "ppo-huggingface",
    organization="HumanCompatibleAI",
    env_name="seals/CartPole-v0",
    venv=venv,
)

/home/chris/miniconda3/envs/env_lift/lib/python3.11/site-packages/stable_baselines3/common/save_util.py:166: UserWarning: Could not deserialize object learning_rate. Consider using `custom_objects` argument to replace this object.
Exception: code() argument 13 must be str, not int
  warnings.warn(
/home/chris/miniconda3/envs/env_lift/lib/python3.11/site-packages/stable_baselines3/common/save_util.py:166: UserWarning: Could not deserialize object clip_range. Consider using `custom_objects` argument to replace this object.
Exception: code() argument 13 must be str, not int
  warnings.warn(
/home/chris/miniconda3/envs/env_lift/lib/python3.11/site-packages/stable_baselines3/common/save_util.py:166: UserWarning: Could not deserialize object lr_schedule. Consider using `custom_objects` argument to replace this object.
Exception: code() argument 13 must be str, not int
  warnings.warn(


Visualization of Expert Policy:

In [2]:
import gymnasium as gym
import os
from IPython.display import display, clear_output
import matplotlib.pyplot as plt

render_mode = "human" if os.environ.get('DISPLAY') else "rgb_array"
env = gym.make("seals:seals/CartPole-v0", render_mode=render_mode)

# Gymnasium returns (obs, info) from reset(); unpack so `obs` is the raw observation
obs, info = env.reset()

for _ in range(1000):
    # Pass only the observation (not the (obs, info) tuple) to the SB3 policy
    action, _ = expert.predict(obs, deterministic=True)

    # Gymnasium step returns (obs, reward, terminated, truncated, info)
    obs, rew, terminated, truncated, info = env.step(action)

    # Render depending on mode. For `rgb_array` we collect and display frames inside the notebook.
    if render_mode == "rgb_array":
        frame = env.render()
        # display every 10 frames to avoid excessive output
        if (_ % 10) == 0:
            clear_output(wait=True)
            plt.imshow(frame)
            plt.axis('off')
            display(plt.gcf())
    else:
        env.render()   # opens a window (when DISPLAY is available)

    # episode end when either terminated or truncated
    if terminated or truncated:
        obs, info = env.reset()
        # if running in rgb_array mode, break after one episode to show the frames
        if render_mode == "rgb_array":
            break

env.close()

/home/chris/miniconda3/envs/env_lift/lib/python3.11/site-packages/pygame/pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


We generate some expert trajectories, that the discriminator needs to distinguish from the learner's trajectories.

In [3]:
from imitation.data import rollout

rollouts = rollout.rollout(
    expert,
    venv,
    rollout.make_sample_until(min_timesteps=None, min_episodes=60),
    rng=np.random.default_rng(SEED),
)

Visualization of sampled Expert Trajectories:

In [4]:
# Exact playback from saved observations (CartPole-specific).
# This will set the environment internal state from each saved observation and render it,
# avoiding stepping actions so visuals match the saved rollouts exactly where possible.
import os
import time
import numpy as np
import gymnasium as gym
from IPython.display import display, clear_output
import matplotlib.pyplot as plt

# Ensure rollouts exist
try:
    trajs = rollouts if hasattr(rollouts, '__len__') else [rollouts]
except NameError:
    raise RuntimeError('`rollouts` not found — run the rollout generation cell first')

render_mode = "human" if os.environ.get('DISPLAY') else "rgb_array"
n_play = min(5, len(trajs))
print(f'Playing {n_play} sampled expert rollouts exactly from saved observations (render_mode={render_mode})')

for i in range(n_play):
    traj = trajs[i]
    # Make a fresh env for each trajectory
    env = gym.make("seals:seals/CartPole-v0", render_mode=render_mode)
    env.reset()

    # Check if we can set internal state directly (CartPole exposes `env.unwrapped.state`).
    can_set_state = hasattr(env, 'unwrapped') and hasattr(env.unwrapped, 'state')
    if not can_set_state:
        print('Warning: env does not support setting internal state; exact playback not possible. Falling back to stepping actions.')

    if can_set_state:
        # traj.obs is length T+1; render each saved observation in sequence
        for t, obs_state in enumerate(traj.obs):
            st = np.asarray(obs_state).copy()
            try:
                env.unwrapped.state = st
            except Exception as e:
                print('Could not assign env.unwrapped.state, falling back to stepping. Error:', e)
                can_set_state = False
                break
            if render_mode == 'rgb_array':
                frame = env.render()
                clear_output(wait=True)
                plt.imshow(frame)
                plt.axis('off')
                display(plt.gcf())
            else:
                env.render()
            # small delay so notebook updates are visible
            time.sleep(0.02)

    if not can_set_state:
        # Fallback: step the saved actions into the env (may diverge from original)
        env.reset()
        for act in traj.acts:
            a = np.asarray(act)
            if a.shape == ():
                a = a.item()
            obs, rew, terminated, truncated, info = env.step(a)
            if render_mode == 'rgb_array':
                frame = env.render()
                clear_output(wait=True)
                plt.imshow(frame)
                plt.axis('off')
                display(plt.gcf())
            else:
                env.render()
            if terminated or truncated:
                break

    env.close()
    # brief pause between trajectories
    if render_mode == 'rgb_array':
        time.sleep(0.5)

print('Done playing exact rollouts')

Playing 5 sampled expert rollouts exactly from saved observations (render_mode=human)


KeyboardInterrupt: 

Now we are ready to set up our AIRL trainer.
Note, that the `reward_net` is actually the network of the discriminator.
We evaluate the learner before and after training so we can see if it made any progress.

Initialization of Learner policy (AIRL Generator):

In [5]:
from imitation.algorithms.adversarial.airl import AIRL
from imitation.rewards.reward_nets import BasicShapedRewardNet
from imitation.util.networks import RunningNorm
from stable_baselines3 import PPO
from stable_baselines3.ppo import MlpPolicy
from stable_baselines3.common.evaluation import evaluate_policy


learner = PPO(
    env=venv,
    policy=MlpPolicy,
    batch_size=64,
    ent_coef=0.0,
    learning_rate=0.0005,
    gamma=0.95,
    clip_range=0.1,
    vf_coef=0.1,
    n_epochs=5,
    seed=SEED,
)


Visualization of untrained Learner Policy (AIRL Generator):

In [6]:
render_mode = "human" if os.environ.get('DISPLAY') else "rgb_array"
env = gym.make("seals:seals/CartPole-v0", render_mode=render_mode)

# Gymnasium returns (obs, info) from reset(); unpack so `obs` is the raw observation
obs, info = env.reset()

for _ in range(1000):
    # Pass only the observation (not the (obs, info) tuple) to the SB3 policy
    action, _ = learner.predict(obs, deterministic=True)

    # Gymnasium step returns (obs, reward, terminated, truncated, info)
    obs, rew, terminated, truncated, info = env.step(action)

    # Render depending on mode. For `rgb_array` we collect and display frames inside the notebook.
    if render_mode == "rgb_array":
        frame = env.render()
        # display every 10 frames to avoid excessive output
        if (_ % 10) == 0:
            clear_output(wait=True)
            plt.imshow(frame)
            plt.axis('off')
            display(plt.gcf())
    else:
        env.render()   # opens a window (when DISPLAY is available)

    # episode end when either terminated or truncated
    if terminated or truncated:
        obs, info = env.reset()
        # if running in rgb_array mode, break after one episode to show the frames
        if render_mode == "rgb_array":
            break

env.close()

KeyboardInterrupt: 

AIRL Training:

In [7]:
reward_net = BasicShapedRewardNet(
    observation_space=venv.observation_space,
    action_space=venv.action_space,
    normalize_input_layer=RunningNorm,
)
airl_trainer = AIRL(
    demonstrations=rollouts,
    demo_batch_size=2048,
    gen_replay_buffer_capacity=512,
    n_disc_updates_per_round=16,
    venv=venv,
    gen_algo=learner,
    reward_net=reward_net,
)

venv.seed(SEED)
learner_rewards_before_training, _ = evaluate_policy(
    learner, venv, 100, return_episode_rewards=True
)
airl_trainer.train(N_RL_TRAIN_STEPS)
venv.seed(SEED)
learner_rewards_after_training, _ = evaluate_policy(
    learner, venv, 100, return_episode_rewards=True
)

round:   0%|          | 0/122 [00:00<?, ?it/s]

------------------------------------------
| raw/                        |          |
|    gen/rollout/ep_len_mean  | 500      |
|    gen/rollout/ep_rew_mean  | 34.4     |
|    gen/time/fps             | 7603     |
|    gen/time/iterations      | 1        |
|    gen/time/time_elapsed    | 2        |
|    gen/time/total_timesteps | 16384    |
------------------------------------------
--------------------------------------------------
| raw/                                |          |
|    disc/disc_acc                    | 0.505    |
|    disc/disc_acc_expert             | 1        |
|    disc/disc_acc_gen                | 0.0103   |
|    disc/disc_entropy                | 0.663    |
|    disc/disc_loss                   | 0.74     |
|    disc/disc_proportion_expert_pred | 0.995    |
|    disc/disc_proportion_expert_true | 0.5      |
|    disc/global_step                 | 1        |
|    disc/n_expert                    | 2.05e+03 |
|    disc/n_generated                 | 2.05e+03 |
-

round:   1%|          | 1/122 [00:04<08:50,  4.39s/it]

-----------------------------------------------------
| raw/                               |              |
|    gen/rollout/ep_len_mean         | 500          |
|    gen/rollout/ep_rew_mean         | 36.7         |
|    gen/rollout/ep_rew_wrapped_mean | -572         |
|    gen/time/fps                    | 7824         |
|    gen/time/iterations             | 1            |
|    gen/time/time_elapsed           | 2            |
|    gen/time/total_timesteps        | 32768        |
|    gen/train/approx_kl             | 0.0009234178 |
|    gen/train/clip_fraction         | 0.026        |
|    gen/train/clip_range            | 0.1          |
|    gen/train/entropy_loss          | -0.692       |
|    gen/train/explained_variance    | -0.0146      |
|    gen/train/learning_rate         | 0.0005       |
|    gen/train/loss                  | 4.4          |
|    gen/train/n_updates             | 5            |
|    gen/train/policy_gradient_loss  | -0.000137    |
|    gen/train/value_loss   

round:   2%|▏         | 2/122 [00:08<08:39,  4.33s/it]

...

round:  98%|█████████▊| 120/122 [08:24<00:08,  4.21s/it]

-----------------------------------------------------
| raw/                               |              |
|    gen/rollout/ep_len_mean         | 500          |
|    gen/rollout/ep_rew_mean         | 500          |
|    gen/rollout/ep_rew_wrapped_mean | -101         |
|    gen/time/fps                    | 7900         |
|    gen/time/iterations             | 1            |
|    gen/time/time_elapsed           | 2            |
|    gen/time/total_timesteps        | 1982464      |
|    gen/train/approx_kl             | 0.0016559655 |
|    gen/train/clip_fraction         | 0.0476       |
|    gen/train/clip_range            | 0.1          |
|    gen/train/entropy_loss          | -0.138       |
|    gen/train/explained_variance    | 0.997        |
|    gen/train/learning_rate         | 0.0005       |
|    gen/train/loss                  | 0.00312      |
|    gen/train/n_updates             | 600          |
|    gen/train/policy_gradient_loss  | 0.00107      |
|    gen/train/value_loss   

round:  99%|█████████▉| 121/122 [08:28<00:04,  4.20s/it]

-----------------------------------------------------
| raw/                               |              |
|    gen/rollout/ep_len_mean         | 500          |
|    gen/rollout/ep_rew_mean         | 500          |
|    gen/rollout/ep_rew_wrapped_mean | -117         |
|    gen/time/fps                    | 7497         |
|    gen/time/iterations             | 1            |
|    gen/time/time_elapsed           | 2            |
|    gen/time/total_timesteps        | 1998848      |
|    gen/train/approx_kl             | 0.0011095797 |
|    gen/train/clip_fraction         | 0.0367       |
|    gen/train/clip_range            | 0.1          |
|    gen/train/entropy_loss          | -0.151       |
|    gen/train/explained_variance    | 0.994        |
|    gen/train/learning_rate         | 0.0005       |
|    gen/train/loss                  | 0.011        |
|    gen/train/n_updates             | 605          |
|    gen/train/policy_gradient_loss  | 0.000833     |
|    gen/train/value_loss   

round: 100%|██████████| 122/122 [08:32<00:00,  4.20s/it]


Visualization of trained Learner Policy (AIRL Generator):

In [8]:
import gymnasium as gym
import os
from IPython.display import display, clear_output
import matplotlib.pyplot as plt

render_mode = "human" if os.environ.get('DISPLAY') else "rgb_array"
env = gym.make("seals:seals/CartPole-v0", render_mode=render_mode)

# Gymnasium returns (obs, info) from reset(); unpack so `obs` is the raw observation
obs, info = env.reset()

for _ in range(1000):
    # Pass only the observation (not the (obs, info) tuple) to the SB3 policy
    action, _ = learner.predict(obs, deterministic=True)

    # Gymnasium step returns (obs, reward, terminated, truncated, info)
    obs, rew, terminated, truncated, info = env.step(action)

    # Render depending on mode. For `rgb_array` we collect and display frames inside the notebook.
    if render_mode == "rgb_array":
        frame = env.render()
        # display every 10 frames to avoid excessive output
        if (_ % 10) == 0:
            clear_output(wait=True)
            plt.imshow(frame)
            plt.axis('off')
            display(plt.gcf())
    else:
        env.render()   # opens a window (when DISPLAY is available)

    # episode end when either terminated or truncated
    if terminated or truncated:
        obs, info = env.reset()
        # if running in rgb_array mode, break after one episode to show the frames
        if render_mode == "rgb_array":
            break

env.close()

KeyboardInterrupt: 

We can see that an untrained policy performs poorly, while AIRL brings an improvement.

In [ ]:
print(
    "Rewards before training:",
    np.mean(learner_rewards_before_training),
    "+/-",
    np.std(learner_rewards_before_training),
)
print(
    "Rewards after training:",
    np.mean(learner_rewards_after_training),
    "+/-",
    np.std(learner_rewards_after_training),
)

Rewards before training: 102.6 +/- 24.11514047232568
Rewards after training: 500.0 +/- 0.0


Training preparation of RL Agent with AIRL Reward Net:

In [10]:
# Use learned reward_net as reward function (SAC if continuous, DQN if discrete).
# The wrapper replaces the environment reward with reward_net.predict_processed(...).
import numpy as np
import gymnasium as gym
from stable_baselines3 import SAC, DQN
from stable_baselines3.sac import MlpPolicy as SACPolicy
from stable_baselines3.dqn import MlpPolicy as DQNPolicy
from stable_baselines3.common.evaluation import evaluate_policy

# Choose the reward net: prefer airl_trainer.reward_test if available
base_reward_net = None
if 'airl_trainer' in globals() and getattr(globals()['airl_trainer'], 'reward_test', None) is not None:
    base_reward_net = globals()['airl_trainer'].reward_test
    print('Using airl_trainer.reward_test as reward_net.')
elif 'reward_net' in globals():
    base_reward_net = globals()['reward_net']
    print('Using reward_net as reward_net.')
else:
    raise RuntimeError('Kein reward_net gefunden. Führe zuerst die AIRL-Zellen aus.')

class RewardNetRewardWrapper(gym.Wrapper):
    """Env wrapper that replaces the environment reward with a learned reward_net.
    Expects observations to be plain ndarrays (not Dict obs).
    """
    def __init__(self, env, reward_net):
        super().__init__(env)
        self.reward_net = reward_net
        self.last_obs = None

    def reset(self, **kwargs):
        res = self.env.reset(**kwargs)
        if isinstance(res, tuple):
            obs, info = res
        else:
            obs, info = res, {}
        self.last_obs = np.asarray(obs)
        return res

    def step(self, action):
        next_obs, _, terminated, truncated, info = self.env.step(action)
        done = bool(terminated or truncated)
        s = np.expand_dims(np.asarray(self.last_obs), 0)
        a = np.expand_dims(np.asarray(action), 0)
        ns = np.expand_dims(np.asarray(next_obs), 0)
        d = np.array([done], dtype=float)
        # use predict_processed if available
        if hasattr(self.reward_net, 'predict_processed'):
            r = float(self.reward_net.predict_processed(s, a, ns, d)[0])
        else:
            r = float(self.reward_net.predict(s, a, ns, d)[0])
        self.last_obs = np.asarray(next_obs)
        return next_obs, r, terminated, truncated, info

# Create base env and wrap
base_env = gym.make('seals:seals/CartPole-v0')
wrapped_env = RewardNetRewardWrapper(base_env, base_reward_net)

# Choose algorithm compatible with action space
# We imported `gymnasium as gym` above; avoid importing the legacy `gym` package.
if isinstance(wrapped_env.action_space, gym.spaces.Box):
    Algo = SAC
    Policy = SACPolicy
    print('Using SAC for continuous action space.')
else:
    # Discrete action spaces -> use DQN
    Algo = DQN
    Policy = DQNPolicy
    print('Using DQN for discrete action space.')

agent = Algo(Policy, wrapped_env, verbose=1, seed=SEED)

Using airl_trainer.reward_test as reward_net.
Using DQN for discrete action space.
Using cuda device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.


Visualization of RL Policy before training:

In [11]:
render_mode = "human" if os.environ.get('DISPLAY') else "rgb_array"
env = gym.make("seals:seals/CartPole-v0", render_mode=render_mode)

# Gymnasium returns (obs, info) from reset(); unpack so `obs` is the raw observation
obs, info = env.reset()

for _ in range(100):
    # Pass only the observation (not the (obs, info) tuple) to the SB3 policy
    action, _ = agent.predict(obs, deterministic=True)

    # Gymnasium step returns (obs, reward, terminated, truncated, info)
    obs, rew, terminated, truncated, info = env.step(action)

    # Render depending on mode. For `rgb_array` we collect and display frames inside the notebook.
    if render_mode == "rgb_array":
        frame = env.render()
        # display every 10 frames to avoid excessive output
        if (_ % 10) == 0:
            clear_output(wait=True)
            plt.imshow(frame)
            plt.axis('off')
            display(plt.gcf())
    else:
        env.render()   # opens a window (when DISPLAY is available)

    # episode end when either terminated or truncated
    if terminated or truncated:
        obs, info = env.reset()
        # if running in rgb_array mode, break after one episode to show the frames
        if render_mode == "rgb_array":
            break

env.close()

KeyboardInterrupt: 

Training RL Agent with AIRL Reward Net:

In [12]:
total_timesteps = 5000000
agent.learn(total_timesteps=total_timesteps)

# Evaluation (return_episode_rewards=True returns (episode_rewards, episode_lengths))
episode_rewards, episode_lengths = evaluate_policy(agent, wrapped_env, n_eval_episodes=10, return_episode_rewards=True)
print('Mean return (10 eps):', np.mean(episode_rewards))

# Save agent
agent.save('agent_with_learned_reward')

-----------------------------------
| rollout/            |           |
|    ep_len_mean      | 500       |
|    ep_rew_mean      | -9.86e+03 |
|    exploration_rate | 0.996     |
| time/               |           |
|    episodes         | 4         |
|    fps              | 3478      |
|    time_elapsed     | 0         |
|    total_timesteps  | 2000      |
-----------------------------------
-----------------------------------
| rollout/            |           |
|    ep_len_mean      | 500       |
|    ep_rew_mean      | -1.32e+04 |
|    exploration_rate | 0.992     |
| time/               |           |
|    episodes         | 8         |
|    fps              | 3485      |
|    time_elapsed     | 1         |
|    total_timesteps  | 4000      |
-----------------------------------
-----------------------------------
| rollout/            |           |
|    ep_len_mean      | 500       |
|    ep_rew_mean      | -1.74e+04 |
|    exploration_rate | 0.989     |
| time/               |     

KeyboardInterrupt: 

Visualize RL Policy, trained with AIRL Reward Net:

In [ ]:
render_mode = "human" if os.environ.get('DISPLAY') else "rgb_array"
env = gym.make("seals:seals/CartPole-v0", render_mode=render_mode)

# Gymnasium returns (obs, info) from reset(); unpack so `obs` is the raw observation
obs, info = env.reset()

for _ in range(1000):
    # Pass only the observation (not the (obs, info) tuple) to the SB3 policy
    action, _ = agent.predict(obs, deterministic=True)

    # Gymnasium step returns (obs, reward, terminated, truncated, info)
    obs, rew, terminated, truncated, info = env.step(action)

    # Render depending on mode. For `rgb_array` we collect and display frames inside the notebook.
    if render_mode == "rgb_array":
        frame = env.render()
        # display every 10 frames to avoid excessive output
        if (_ % 10) == 0:
            clear_output(wait=True)
            plt.imshow(frame)
            plt.axis('off')
            display(plt.gcf())
    else:
        env.render()   # opens a window (when DISPLAY is available)

    # episode end when either terminated or truncated
    if terminated or truncated:
        obs, info = env.reset()
        # if running in rgb_array mode, break after one episode to show the frames
        if render_mode == "rgb_array":
            break

env.close()